In [1]:
import sys
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/src/')
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/models/')
sys.path.append('/home/rrg0013@auburn.edu/git/nodules/keras/')

In [2]:
import fpr_xrv2_1c3 as m
import generatorModel as gen
import kerasUtil as ku
from sklearn.metrics import roc_curve, f1_score, auc
from glob import glob

Using TensorFlow backend.
Using cuDNN version 6020 on context None
Mapped name None to device cuda0: GeForce GTX 1070 (0000:83:00.0)


In [3]:
import os
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt

### Step 1

In [9]:
PATH = '/scr/data/nodules/testseg/cands/train2/seven/'
#PATH = '/scr/data/nodules/testseg/cands/test3a/'
#WEIGHT = '/scr/data/nodules/models/five_fpr1.h5'
#PATH = '/scr/data/nodules/testseg/cands/train2/'+NO_+'/'
WEIGHT = '/scr/data/nodules/models/four_fpr1.h5'
WEIGHT = '/scr/data/nodules/init3dConv/073117/7003/64-no7003-accuracy-073117.h5'

### Step 2

In [10]:
PATH = '/scr/data/nodules/testseg/cands/train4/seven/'
#PATH = '/scr/data/nodules/testseg/cands/train3/nine/'
WEIGHT = '/scr/data/nodules/models/three_fpr1.h5'
WEIGHT = '/scr/data/nodules/init3dConv/073117/7003/64-no7003-accuracy-073117.h5'

In [881]:
PATH = '/scr/data/nodules/testseg/cands/train2/seven/'
PATH = '/scr/data/nodules/testseg/cands/train3/seven/'
#WEIGHT = '/scr/data/nodules/init3dConv/080217/1001/32-no1001-accuracy-080217.h5'
#PATH = '/scr/data/nodules/testseg/cands/test3/'
PATH = '/scr/data/nodules/testseg/final/final/clean/f/zero/'
#PATH = '/scr/data/nodules/testseg/final/finalt/clean/t/zero/'
WEIGHT = '/scr/data/nodules/init3dConv/080217/1001/133.h5'

In [882]:
def loadpng1(cfg, full):
  path = cfg
  images, labels = [], []
  for parent, subdir, files in os.walk(path):
    for file in files:
      image = scipy.misc.imread(os.path.join(parent,file))
      image = ku.formatImage(image,full)
      images.append(np.array(image))
      label = file
      if label[1] == '.':
        label = label[2:]
      elif label[2] == '.':
        label = label[3:]
      label = str(label)
      meta = label.split('_')
      label = {'labels':meta[0]}
      #print label
      #label['malignancy'] = meta[1].split('-')
      #label['vars'] = meta[2].split('-')
      #meta = meta[2].split('-')
      #print meta
      #meta = meta[7].split('.')
      #label['count'] = meta[3]
      #label['slice'] = meta[4]
      labels.append(label)
  if path[-4:] == '.png':
    image = scipy.misc.imread(path)
    image = ku.formatImage(image,full)
    images.append(np.array(image))
    label = path.split('/')
    label = label[-1]
    if label[1] == '.':
      label = label[2:]
    elif label[2] == '.':
      label = label[3:]
    label = str(label)
    meta = label.split('_')
    label = {'labels':meta[0]}
    labels.append(label)
  return images, labels

In [883]:
#images, labels = loadpng1(PATH, True)
images, labels = loadpng1(PATH, True)
images, labels = ku.cleanData(images, labels, 2)
images = ku.to3d(images, 2, True)
labels = ku.get2Labels(labels)
images = ku.convertArray(images)

In [884]:
len(images)

3148

In [885]:
print len(images)

3148


In [886]:
#std = np.std(images)+1e-12
#mean = np.mean(images)+1e-12
mean = 0.191494569183
#0.2289698869#0.238903835417
std = 0.197447970511
#0.218540757896#0.221545860172
mean=0.190817147494
std=0.199062317611
mean=0.188313096763
std=0.1979406029

images -= .17#mean#0.237466380001#0.239519521595
images /= .17#std#0.221488147975#0.221684783698
#images -= mean
#images /= std
#print mean
#print std

In [887]:
0.237466380001
0.221488147975

0.221488147975

In [888]:
model = m.getModel(2,sz=32,full=True)

62
31
52
26
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_67 (InputLayer)            (None, 1, 32, 32, 32) 0                                            
____________________________________________________________________________________________________
conv3d_3565 (Conv3D)             (None, 32, 32, 32, 32 896         input_67[0][0]                   
____________________________________________________________________________________________________
conv3d_3570 (Conv3D)             (None, 6, 32, 32, 32) 198         conv3d_3565[0][0]                
____________________________________________________________________________________________________
conv3d_3566 (Conv3D)             (None, 6, 32, 32, 32) 198         conv3d_3565[0][0]                
_______________________________________________________________________________

In [889]:
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [890]:
model.load_weights(WEIGHT)

In [891]:
preds = model.predict(images, batch_size=64)

In [879]:
fpr, tpr, _ = roc_curve(labels[:,0],preds[:,0])
area = auc(fpr, tpr)
print area
plt.plot(fpr,tpr)
plt.savefig('/home/rrg0013@auburn.edu/four_roc.png')  
plt.clf()

nan


### Found of all candidate nodules -- Step 1

In [892]:
tmp = np.zeros(preds.shape[0])
tmp[preds[:,0] > preds[:,1]] = 1
pos = np.sum(tmp)
print pos

188.0


### Found of all candidate nodules with at least one large label -- Step 2

In [829]:
tmp = np.zeros(preds.shape[0])
tmp[preds[:,0] > preds[:,1]] = 1
xpos = np.sum(tmp)
print xpos

98.0


### False positives per scan -- Step 2

In [712]:
perscan = (pos-xpos)/89. 
print perscan

1.95505617978


### Sensitivity -- Step 1

In [697]:
tlbl = np.zeros(preds.shape[0])
tlbl[labels[:,0]==1] = 1
tp_fn = np.sum(tlbl)
print tp_fn

107.0


In [698]:
tmp2 = np.zeros(preds.shape[0])
tmp2[preds[:,0] > preds[:,1]] = 1
tmp2[tlbl==1] += 1
tmp3 = np.where(tmp2==2,1,0)
tp = np.sum(tmp3)
print tp

100


#### SENSITIVITY VALUE:

In [699]:
sensitivity = tp/tp_fn
print sensitivity

0.934579439252


### Test

In [453]:
print (pos-xpos-tp)/89.

-0.370786516854


### Specificity -- Step 1

In [404]:
tlbl = np.zeros(preds.shape[0])
tlbl[labels[:,1]==1] = 1
tn_fp = np.sum(tlbl)
print tn_fp

6222.0


In [405]:
tmp = np.zeros(preds.shape[0])
tmp[preds[:,1] > preds[:,0]] = 1
tn = np.sum(tmp)
print tn

5891.0


### Specificity -- Step 2

In [427]:
tn_fp = tn_fp-(xpos-tp)
print tn_fp

2813.0


#### SPECIFICITY VALUE:

In [428]:
specificity = tn/tn_fp
print specificity

0.941343761109


In [ ]:
ct = 0
paths = glob(PATH+'*.png')
for i, path in enumerate(paths):
  TPATH = path
  #print TPATH
  image, label = loadpng1(TPATH, True)
  #print len(image)
  image, label = ku.cleanData(image, label, 2)
  image = ku.to3d(image, 0, True)
  label = ku.get2Labels(label)
  image = ku.convertArray(image)
  pred = model.predict(image,batch_size=1)
  if pred[0,1]>pred[0,0]:
    ct += 1
    print pred[0,1]
    print pred[0,0]
    #print path
print ct